In [1]:
from imbd.data import DataLoader, DataPreprocessor
import numpy as np
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_validate

In [2]:
loader = DataLoader()

In [3]:
df = loader.build_label_20_df()

In [4]:
preprocessor = DataPreprocessor()

In [5]:
train_features = df.drop(loader.labels, axis=1)
train_labels = df[loader.labels]

In [6]:
train_features = preprocessor(train_features)

[Pipeline] .. (step 1 of 3) Processing drop_na_features, total=   0.0s
[Pipeline] ...... (step 2 of 3) Processing quantization, total=   0.2s
[Pipeline] ........... (step 3 of 3) Processing fill_na, total=   0.2s


In [7]:
model = MultiOutputRegressor(XGBRegressor())
scorer = make_scorer(mean_squared_error, squared=False)

In [8]:
res = cross_validate(model, train_features, train_labels, cv=10, scoring=scorer, return_estimator=True)

In [11]:
res['test_score']

array([0.09220902, 0.15879338, 0.14849221, 0.18385251, 0.2209088 ,
       0.10375198, 0.1771144 , 0.07285427, 0.07387997, 0.08370213])

In [9]:
best_estimator = res['estimator'][np.argmax(res['test_score'])]

In [10]:
pred = best_estimator.predict(train_features)

In [24]:
res = np.abs(pred - train_labels)
res['mean'] = np.mean(res, axis=1)

In [25]:
res.sort_values('mean', ascending=False).to_csv('output.csv', index=False)